In [3]:
# Biblioteki wejściowe
import pandas as pd
import numpy as np
import posixpath as os
import pathlib
from pathlib import Path
import pyodbc
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
#pd.options.display.min_colwidth=10
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

DATA = Path(r'/Users/daniel_wojcik/Documents/LIBERTAS/FINANSE/SF 2022/DATA')
ASSETS = Path(r'/Users/daniel_wojcik/Documents/LIBERTAS/FINANSE/SF 2022/ASSETS')
IMAGES = Path(r'/Users/daniel_wojcik/Documents/LIBERTAS/FINANSE/SF 2022/IMAGES')

msc=12

In [3]:
# ZAPYTANIE SQL

t_sql="SELECT Konta.Symbol, Konta.Nazwa, ZapisyKsiegowe.NumerDokumentu,ZapisyKsiegowe.Opis,ZapisyKsiegowe.Data, ZapisyKsiegowe.Strona, ZapisyKsiegowe.KwotaZapisuValue FROM ZapisyKsiegowe INNER JOIN Konta ON ZapisyKsiegowe.Konto = Konta.ID AND ZapisyKsiegowe.Okres = Konta.Okres WHERE (ZapisyKsiegowe.Data >= CONVERT(DATETIME, '2022-01-01 00:00:00', 102))"

cnxn=pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                    'Server=192.168.1.252\ENOVA_KOPALNIA;'
                    'Database=LIBERTAS;'
                    'UID=ro_daniel.wojcik;'
                    'PWD=661594050xkr@;'
                    'Trusted_Connection=no;')

t_zapisy=pd.read_sql_query (t_sql,cnxn)
t_zapisy.to_csv(DATA /'SF_Zapisy_SQL.csv',index=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
# Pobiera dane wejściowe MAC

t_zapisy=pd.read_csv(DATA /'SF_Zapisy_SQL.csv')


In [5]:
# Pobiera wykonanie z zapisów księgowych

t_zapisy['Data']=pd.to_datetime(t_zapisy['Data'], infer_datetime_format=True)

t_zapisy['Kwota_DT']=t_zapisy.apply(lambda x:x['KwotaZapisuValue'] if x['Strona']!=1 else 0,axis=1)
t_zapisy['Kwota_CR']=t_zapisy.apply(lambda x:x['KwotaZapisuValue'] if x['Strona']==1 else 0,axis=1)
t_zapisy['Kwota_SAL']=t_zapisy['Kwota_DT']-t_zapisy['Kwota_CR']


t_zapisy.drop(['Strona'],axis=1,inplace=True)
t_zapisy.drop(['KwotaZapisuValue'],axis=1,inplace=True)

t_zapisy.rename(columns={"Symbol":'Konto','NumerDokumentu':'N_Dokument'},inplace=True)
t_zapisy['Data_M']=t_zapisy['Data']+MonthEnd(0)
t_zapisy['R']=t_zapisy['Data'].dt.year
t_zapisy['M']=t_zapisy['Data'].dt.month
t_zapisy['ID']=0
t_zapisy=t_zapisy[t_zapisy['Konto'].str.startswith('9')==False]

In [10]:
t_zapisy.head()

,Konto,Nazwa,N_Dokument,Opis,Data,Kwota_DT,Kwota_CR,Kwota_SAL,Data_M,R,M,ID
0,010-7,ŚRODKI TRANSPORTU,OT/2022/11/0001,OT/2022/11/0001 Zakup,2022-11-16,0.00,"67,081.30","-67,081.30",2022-11-30,2022,11,0
1,080-001,ŚRODKI TRWAŁE NIE WYMAGAJĄCE MONTAŻU,FV/22/11/016,ZAKUP SAMOCHODU SKODA OCTAVIA WI711JU ZARZĄD,2022-11-16,0.00,"60,162.60","-60,162.60",2022-11-30,2022,11,0
2,080-001,ŚRODKI TRWAŁE NIE WYMAGAJĄCE MONTAŻU,FV/22/11/016,ZAKUP SAMOCHODU SKODA OCTAVIA WI711JU ZARZĄD,2022-11-16,0.00,"6,918.70","-6,918.70",2022-11-30,2022,11,0
3,080-001,ŚRODKI TRWAŁE NIE WYMAGAJĄCE MONTAŻU,OT/2022/11/0001,OT/2022/11/0001 Zakup,2022-11-16,"67,081.30",0.00,"67,081.30",2022-11-30,2022,11,0
4,149-1,ŚRODKI PIENIĘŻNE W DRODZE,WB/PODST/2022/041,Przeksięg.środków na rach.powiązany [2],2022-11-21,0.00,"1,689.00","-1,689.00",2022-11-30,2022,11,0


In [14]:
t_konta=t_zapisy[['Konto','Nazwa']].copy()
t_konta.drop_duplicates(inplace=True)

In [16]:
t_konta.sort_values(by='Konto')

,Konto,Nazwa
0,010-7,ŚRODKI TRANSPORTU
1,080-001,ŚRODKI TRWAŁE NIE WYMAGAJĄCE MONTAŻU
343,130-001,BIEŻĄCY RACHUNEK BANKOWY PEKAO
497,130-004,Płacowy
702,132-1,LOKATY TERMINOWE PEKAO SA
4,149-1,ŚRODKI PIENIĘŻNE W DRODZE
744,202-GRAUSCH I GRAUSCH SP,GRAUSCH I GRAUSCH MASZYNY BUDOWLANE SPÓŁKA Z OGRANICZONĄ ODPOWIED...
622,202-GRUPA INCO,"""GRUPA INCO"" SPÓŁKA AKCYJNA"
712,202-KAN.RADCY PRAWNEGO,Kancelaria Radcy Prawnego Piotr Szeląg
749,202-KIESEL POLAND,KIESEL POLAND SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ


## Bilans - aktywa

In [ ]:
# WNIP

t_zapisy['ID']=t_zapisy.apply(lambda x:'A1' if (
    (x['Konto'].startswith('020'))
    or (x['Konto'].startswith('072'))
    )
else x['ID'],axis=1)

In [ ]:
# Grunty

t_zapisy['ID']=t_zapisy.apply(lambda x:'A2' if (
    (x['Konto'].startswith('010-001'))
    or (x['Konto'].startswith('010-002'))
    or (x['Konto'].startswith('010-003'))
    or (x['Konto'].startswith('071-000'))
    or (x['Konto'].startswith('071-002'))
    )
else x['ID'],axis=1)

In [ ]:
# Budynki

t_zapisy['ID']=t_zapisy.apply(lambda x:'A3' if (
    (x['Konto'].startswith('010-100'))
    or (x['Konto'].startswith('010-200'))
    or (x['Konto'].startswith('071-100'))
    or (x['Konto'].startswith('071-200'))
    )
else x['ID'],axis=1)

In [ ]:
# Urządzenia techniczne i maszyny

t_zapisy['ID']=t_zapisy.apply(lambda x:'A4' if (
    (x['Konto'].startswith('010-400'))
    or (x['Konto'].startswith('010-500'))
    or (x['Konto'].startswith('010-600'))
    or (x['Konto'].startswith('071-400'))
    or (x['Konto'].startswith('071-500'))
    or (x['Konto'].startswith('071-600'))
    )
else x['ID'],axis=1)

In [ ]:
# Środki transportu

t_zapisy['ID']=t_zapisy.apply(lambda x:'A5' if (
    (x['Konto'].startswith('010-700'))
    or (x['Konto'].startswith('071-700'))
    )
else x['ID'],axis=1)

In [ ]:
# Inne środki trwałe

t_zapisy['ID']=t_zapisy.apply(lambda x:'A6' if (
    (x['Konto'].startswith('010-800'))
    or (x['Konto'].startswith('071-800'))
    )
else x['ID'],axis=1)

In [ ]:
# Środki trwałe w budowie

t_zapisy['ID']=t_zapisy.apply(lambda x:'A7' if (
    (x['Konto'].startswith('080'))
    )
else x['ID'],axis=1)

In [ ]:
# Aktywa z odroczonego podatku dochodowego

t_zapisy['ID']=t_zapisy.apply(lambda x:'A10' if (
    (x['Konto'].startswith('650-001'))
    )
else x['ID'],axis=1)

In [ ]:
# Inne rozliczenia międzyokresowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'A11' if (
    (x['Konto'].startswith('645'))
    )
else x['ID'],axis=1)

In [ ]:
# Zapasy - materiały

t_zapisy['ID']=t_zapisy.apply(lambda x:'A12' if (
    (x['Konto'].startswith('311'))
    or (x['Konto'].startswith('310'))
    or (x['Konto'].startswith('300'))
    )
else x['ID'],axis=1)

In [ ]:
# Zapasy - produkty gotowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'A13' if (
    (x['Konto'].startswith('600'))
    )
else x['ID'],axis=1)

In [ ]:
# Należności od pozostałych jednostek - z tytułu dostaw i usług

t_zapisy['ID']=t_zapisy.apply(lambda x:'A15' if (
    (x['Konto'].startswith('200'))
    or (x['Konto'].startswith('280'))
    )
else x['ID'],axis=1)

In [ ]:
# Należności z tytułu podatków

t_zapisy['ID']=t_zapisy.apply(lambda x:'A16' if (
    (x['Konto'].startswith('220')) # tylko strona WN
    or (x['Konto'].startswith('221'))
    or (x['Konto'].startswith('222'))
    )
else x['ID'],axis=1)

In [ ]:
# Należności inne

t_zapisy['ID']=t_zapisy.apply(lambda x:'A17' if (
    (x['Konto'].startswith('234'))
    #or (x['Konto'].startswith('220')) # tylko strona MA
    or (x['Konto'].startswith('249-100-01'))
    or (x['Konto'].startswith('249-100-09'))
    )
else x['ID'],axis=1)

In [ ]:
# Środki pieniężne w kasie i na rachunkach

t_zapisy['ID']=t_zapisy.apply(lambda x:'A18' if (
    (x['Konto'].startswith('100'))
    or (x['Konto'].startswith('130'))
    or (x['Konto'].startswith('131'))
    or (x['Konto'].startswith('140'))
    )
else x['ID'],axis=1)

In [ ]:
# Krótkoterminowe rozliczenia RMK

t_zapisy['ID']=t_zapisy.apply(lambda x:'A19' if (
    (x['Konto'].startswith('640'))
    or (x['Konto'].startswith('641'))
    or (x['Konto'].startswith('646'))
    or (x['Konto'].startswith('644'))
    )
else x['ID'],axis=1)

## Bilans - Pasywa

In [ ]:
# Kapitał podstawowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P1' if (
    (x['Konto'].startswith('800'))
    )
else x['ID'],axis=1)

In [ ]:
# Kapitał zapasowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P2' if (
    (x['Konto'].startswith('811'))
    )
else x['ID'],axis=1)

In [ ]:
# Kapitał z aktualizacji

t_zapisy['ID']=t_zapisy.apply(lambda x:'P3' if (
    (x['Konto'].startswith('814'))
    )
else x['ID'],axis=1)

In [ ]:
# Kapitał rezerwowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P4' if (
    (x['Konto'].startswith('812'))
    )
else x['ID'],axis=1)

In [ ]:
# Kapitał rezerwowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P5' if (
    (x['Konto'].startswith('860'))
    )
else x['ID'],axis=1)

In [ ]:
# Odpisy z zysku w ciągu roku

t_zapisy['ID']=t_zapisy.apply(lambda x:'P6' if (
    (x['Konto'].startswith('820'))
    )
else x['ID'],axis=1)

In [ ]:
# Rezerwa z tytułu odroczonego podatku dochodowego

t_zapisy['ID']=t_zapisy.apply(lambda x:'P7' if (
    (x['Konto'].startswith('830'))
    )
else x['ID'],axis=1)

In [ ]:
# Rezerwa na świadczenia emerytalne

t_zapisy['ID']=t_zapisy.apply(lambda x:'P8' if (
    (x['Konto'].startswith('642-002'))
    or (x['Konto'].startswith('642-005'))
    or (x['Konto'].startswith('642-001'))
    or (x['Konto'].startswith('642-003'))
    or (x['Konto'].startswith('642-004'))
    or (x['Konto'].startswith('642-006'))
    )
else x['ID'],axis=1)

In [ ]:
# Pozostałe rezerwy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P9' if (
    (x['Konto'].startswith('643-001-02'))
    or (x['Konto'].startswith('640-015'))
    or (x['Konto'].startswith('830-200'))
    or (x['Konto'].startswith('643-001-01'))
    )
else x['ID'],axis=1)

In [ ]:
# Kredyt i pożyczki długoterminowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'P10' if (
    (x['Konto'].startswith('135'))
    or (x['Konto'].startswith('241'))
    )
else x['ID'],axis=1)

In [ ]:
# Kredyt i pożyczki krótkoterminowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'P13' if (
    (x['Konto'].startswith('134'))
    or (x['Konto'].startswith('240'))
    )
else x['ID'],axis=1)

In [ ]:
# Zobowiązania krótkoterminowe - dostawy i usługi

t_zapisy['ID']=t_zapisy.apply(lambda x:'P14' if (
    (x['Konto'].startswith('210'))
    )
else x['ID'],axis=1)

In [ ]:
# Zobowiązania krótkoterminowe - podatki

t_zapisy['ID']=t_zapisy.apply(lambda x:'P15' if (
    (x['Konto'].startswith('223-100'))
    or (x['Konto'].startswith('223-200'))
    or (x['Konto'].startswith('223-300'))
    or (x['Konto'].startswith('223-400'))
    or (x['Konto'].startswith('219'))
    )
else x['ID'],axis=1)

In [ ]:
# Zobowiązania krótkoterminowe - wynagrodzenia

t_zapisy['ID']=t_zapisy.apply(lambda x:'P16' if (
    (x['Konto'].startswith('230'))
    )
else x['ID'],axis=1)

In [ ]:
# Zobowiązania krótkoterminowe - inne

t_zapisy['ID']=t_zapisy.apply(lambda x:'P17' if (
    (x['Konto'].startswith('249'))
    )
else x['ID'],axis=1)

In [ ]:
# Fundusze specjalne

t_zapisy['ID']=t_zapisy.apply(lambda x:'P18' if (
    (x['Konto'].startswith('850'))
    or (x['Konto'].startswith('851'))
    )
else x['ID'],axis=1)

In [ ]:
# Rozliczenia międzyokresowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'P19' if (
    (x['Konto'].startswith('840-100'))
    )
else x['ID'],axis=1)

## Rachunek wyników

In [ ]:
# Przychody ze sprzedaży

t_zapisy['ID']=t_zapisy.apply(lambda x:'R1' if (
    (x['Konto'].startswith('700'))
    or (x['Konto'].startswith('703'))
    )
else x['ID'],axis=1)

In [ ]:
# Zmiana stanu produktów

t_zapisy['ID']=t_zapisy.apply(lambda x:'R4' if (
    (x['Konto'].startswith('701-100'))
    or (x['Konto'].startswith('490-02'))
    or (x['Konto'].startswith('791-100'))
    )
else x['ID'],axis=1)

In [ ]:
# Koszt wytworzenia produktów na własne potrzeby

t_zapisy['ID']=t_zapisy.apply(lambda x:'R3' if (
    (x['Konto'].startswith('790')) 
    )
else x['ID'],axis=1)

In [ ]:
# Przychody ze sprzedaży towarów i materiałów

t_zapisy['ID']=t_zapisy.apply(lambda x:'R2' if (
    (x['Konto'].startswith('730'))
    or (x['Konto'].startswith('740'))
    )
else x['ID'],axis=1)

In [ ]:
# Koszty rodzajowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'R5' if ((x['Konto'].startswith('400'))) else x['ID'],axis=1) # amortyzacja
t_zapisy['ID']=t_zapisy.apply(lambda x:'R6' if ((x['Konto'].startswith('401'))) else x['ID'],axis=1) # zużycie materiałów i energii
t_zapisy['ID']=t_zapisy.apply(lambda x:'R7' if ((x['Konto'].startswith('402'))) else x['ID'],axis=1) # usługi obce
t_zapisy['ID']=t_zapisy.apply(lambda x:'R8' if ((x['Konto'].startswith('403'))) else x['ID'],axis=1) # podatki i opłaty
t_zapisy['ID']=t_zapisy.apply(lambda x:'R9' if ((x['Konto'].startswith('404'))) else x['ID'],axis=1) # Wynagrodzenia
t_zapisy['ID']=t_zapisy.apply(lambda x:'R10' if ((x['Konto'].startswith('405'))) else x['ID'],axis=1) # ZUS
t_zapisy['ID']=t_zapisy.apply(lambda x:'R11' if ((x['Konto'].startswith('409'))) else x['ID'],axis=1) # pozostałe

In [ ]:
# Wartość sprzedanych towarów i materiałów

t_zapisy['ID']=t_zapisy.apply(lambda x:'R12' if (
    (x['Konto'].startswith('731'))
    or (x['Konto'].startswith('741'))
    )
else x['ID'],axis=1)

In [ ]:
# Zysk ze sprzedaży MT

t_zapisy['ID']=t_zapisy.apply(lambda x:'R13' if (
    (x['Konto'].startswith('760-002'))
    or (x['Konto'].startswith('761-011'))
    )
else x['ID'],axis=1)

In [ ]:
# Inne przychody operacyjne

t_zapisy['ID']=t_zapisy.apply(lambda x:'R14' if (
    (x['Konto'].startswith('760-001'))
    or (x['Konto'].startswith('760-003'))
    or (x['Konto'].startswith('760-013'))
    or (x['Konto'].startswith('760-005'))
    or (x['Konto'].startswith('761-013'))
    or (x['Konto'].startswith('760-018'))
    or (x['Konto'].startswith('760-006'))
    or (x['Konto'].startswith('760-008'))
    or (x['Konto'].startswith('760-011'))
    or (x['Konto'].startswith('760-015'))
    or (x['Konto'].startswith('750-001-02'))
    or (x['Konto'].startswith('760-004'))
    or (x['Konto'].startswith('760-019'))

    )
else x['ID'],axis=1)

In [ ]:
# Inne koszty operacyjne

t_zapisy['ID']=t_zapisy.apply(lambda x:'R15' if (
    (x['Konto'].startswith('761-001'))
    or (x['Konto'].startswith('761-010'))
    or (x['Konto'].startswith('761-012'))
    or (x['Konto'].startswith('761-014'))
    or (x['Konto'].startswith('761-022'))
    or (x['Konto'].startswith('761-023'))
    or (x['Konto'].startswith('761-024'))
    or (x['Konto'].startswith('760-007'))
    or (x['Konto'].startswith('761-009-03'))
    or (x['Konto'].startswith('761-009-04'))
    or (x['Konto'].startswith('761-003'))
    
    )
else x['ID'],axis=1)

In [ ]:
# Koszty finansowe - odsetki

t_zapisy['ID']=t_zapisy.apply(lambda x:'R17' if (
    (x['Konto'].startswith('751-001'))
    or (x['Konto'].startswith('751-003'))
    )
else x['ID'],axis=1)

In [ ]:
# Koszty finansowe - inne

t_zapisy['ID']=t_zapisy.apply(lambda x:'R18' if (
    (x['Konto'].startswith('751-002'))
    or (x['Konto'].startswith('751-004'))
    )
else x['ID'],axis=1)

In [ ]:
# Podatek dochodowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'R19' if (
    (x['Konto'].startswith('870'))
    )
else x['ID'],axis=1)

In [ ]:
# Obrotówka
t_balance=t_zapisy.groupby(['Konto','Nazwa','M','ID'],as_index=False)[['Kwota_DT','Kwota_CR','Kwota_SAL']].sum()
t_balance[t_balance['ID']==0]

In [ ]:
t_zapisy[t_zapisy['ID']==0]

In [ ]:
t_zapisy.to_csv(DATA /'SF_Zapisy_RAP.csv',index=False)
t_balance.to_csv(DATA /'SF_Balance_RAP.csv',index=False)